# This is all over the place, don't worry will be fixed

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import os
import sys
module_path = os.path.abspath(os.path.join(os.pardir, os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

import nltk
from nltk.probability import FreqDist
from nltk.corpus import stopwords
from nltk.tokenize import regexp_tokenize, word_tokenize, RegexpTokenizer
from nltk import pos_tag
from nltk.corpus import wordnet
import matplotlib.pyplot as plt
import string
import re

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer,\
HashingVectorizer
%matplotlib inline

Loading in the data

In [2]:
df= pd.read_csv('CraigslistCars.csv')

In [3]:
df_1 = df.copy()

In [ ]:
df_1.head()
# visually seeing the first 5 rows of the raw dataset

From the first few lines we are seeing that there are many rows that have NaN as a value within a column.

In [ ]:
# Will take a random sample of a row to see if an arbritrary row will also follow suit of the first 5 rows by having 
# multiple NaN values
df_1.sample()

As you may possibly see above at least one column could have a NaN value in a row.

In [ ]:
# Want to see what are the 25 columns that will be dealt with. Also will see which should be the target for this dataset
df_1.columns

Seeing that within the dataset that price exists and therefore should be a solid target to predict.

In [ ]:
# Seeing the range of the prices
df_1.price.min(), df_1.price.max()

In [ ]:
# The range of price within the dataset goes from 0 to 3,736,928,711
# Want to check out the years of these vehicles because some older cars have been valued in the millions
df_1.loc[df_1['price'] == df_1.price.min(), ['year']], df_1.loc[df_1['price']==df_1.price.max(), ['year']]

Seeing that over 30,000 rows have the price of 0 and 2 have the large value it is showing the dataset is far fromm perfect

Best way to have the price of 0 not sway modeling later on is to limit price values as well as these strange values that are in the billions. Therefore will take an inner 90% of the data.

In [ ]:
df_1['price'] = df_1['price'][df_1['price'].between(df_1['price'].quantile(.085), df_1['price'].quantile(.985),inclusive=True)]
df_1.price.min(), df_1.price.max()
# shows 100 - 59999 is an inner 90%%

In [ ]:
# Want new df with just price values that range from 100 to 59999

In [4]:
# Making new df of just values of cars between 100 and 59,999
df_2 = df_1.loc[(df_1['price']>100) & (df_1['price'] <59999)].copy()
df_2.head(), len(df_2)

(           id                                                url  \
 0  7222695916  https://prescott.craigslist.org/cto/d/prescott...   
 1  7218891961  https://fayar.craigslist.org/ctd/d/bentonville...   
 2  7221797935  https://keys.craigslist.org/cto/d/summerland-k...   
 3  7222270760  https://worcester.craigslist.org/cto/d/west-br...   
 4  7210384030  https://greensboro.craigslist.org/cto/d/trinit...   
 
                    region                         region_url  price  year  \
 0                prescott    https://prescott.craigslist.org   6000   NaN   
 1            fayetteville       https://fayar.craigslist.org  11900   NaN   
 2            florida keys        https://keys.craigslist.org  21000   NaN   
 3  worcester / central MA   https://worcester.craigslist.org   1500   NaN   
 4              greensboro  https://greensboro.craigslist.org   4900   NaN   
 
   manufacturer model condition cylinders  ... size  type paint_color  \
 0          NaN   NaN       NaN       NaN

# Dealing with all the possible NaN values in the dataset

In [ ]:
df_2.isna().sum()

year, manufacturer, model, condition, cylinders, fuel, odometer, title_status, transmission, VIN, drive, size, type, paint_color
image_url, description, county, state, lat, long, posting_date have Na values and some have around 40% of its rows with NaN values.

In [5]:
# Checking the 'county' column since it has the most NaN values.
df_2['county'].value_counts()
# is completely empty

Series([], Name: county, dtype: int64)

In [6]:
# The column is not going to be useful therefore it is safe to just drop it
df_2.drop('county', axis=1, inplace=True)
df_2.head()

,id,url,region,region_url,price,year,manufacturer,model,condition,cylinders,...,drive,size,type,paint_color,image_url,description,state,lat,long,posting_date
0,7222695916,https://prescott.craigslist.org/cto/d/prescott...,prescott,https://prescott.craigslist.org,6000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,az,NaN,NaN,NaN
1,7218891961,https://fayar.craigslist.org/ctd/d/bentonville...,fayetteville,https://fayar.craigslist.org,11900,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,ar,NaN,NaN,NaN
2,7221797935,https://keys.craigslist.org/cto/d/summerland-k...,florida keys,https://keys.craigslist.org,21000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,fl,NaN,NaN,NaN
3,7222270760,https://worcester.craigslist.org/cto/d/west-br...,worcester / central MA,https://worcester.craigslist.org,1500,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,ma,NaN,NaN,NaN
4,7210384030,https://greensboro.craigslist.org/cto/d/trinit...,greensboro,https://greensboro.craigslist.org,4900,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,nc,NaN,NaN,NaN


County dealt with

The 'year' column is the first column that shows it has NaN values so lets take a deeper dive into the column

In [ ]:
df_2['year'].value_counts()

In [ ]:
# Checking the oldest car
df_2['year'].min()

In [ ]:
# Checking newest car
df_2.year.max()

In [ ]:
# Making a sorted list of all the years of the cars represented in the dataset
unique_years = df_2['year'].unique()
print(sorted(unique_years))

Barring a couple of years in the early 1900's, the dataset seems to have nearly every year in the 20th century and all years that have occured in the 21st century.

In [ ]:
type(unique_years)

In [ ]:
df_2.year.value_counts(normalize=True)

In [ ]:
df_2.year.value_counts()

In [ ]:
df_top_years = df_2.copy()
df_top_years['year'] = df_top_years['year'][df_top_years['year'].between(df_top_years['year'].quantile(.1), df_top_years['year'].quantile(1),inclusive=True)]
df_top_years.year.min(), df_top_years.year.max()

In [7]:
df_limited_years= df_2.loc[df_2['year'] >2002].copy()

In [ ]:
df_limited_years.year.value_counts()

In [ ]:
df_limited_years.isna().sum()

In [8]:
df_condition = df_limited_years.loc[df_limited_years['condition'].notna()].copy()
df_model_and_condition = df_condition.loc[df_condition['model'].notna()].copy()
condition_dict = {}
for a, b in df_model_and_condition[['model', 'condition']].values:
    condition_dict[a] = b

In [9]:
df_limited_years['condition'] = df_limited_years.apply(lambda x: x['condition'] if pd.isna(x['model']) or pd.notna(x['condition']) else condition_dict[x['model']] if x['model'] in condition_dict.keys() else x['condition'], axis=1)

In [10]:
df_limited_years.isna().sum()

id                   0
url                  0
region               0
region_url           0
price                0
year                 0
manufacturer     10234
model             2530
condition        17764
cylinders       144503
fuel              2226
odometer          2050
title_status      7004
transmission      1645
VIN             120074
drive           105630
size            253876
type             69704
paint_color     101144
image_url            0
description          2
state                0
lat               3283
long              3283
posting_date         0
dtype: int64

In [11]:
df_cylinders = df_limited_years.loc[df_limited_years['cylinders'].notna()].copy()
df_cylinders_model = df_cylinders.loc[df_cylinders['model'].notna()].copy()
cylinders_dictionary = {}
for c, d in df_cylinders_model[['model', 'cylinders']].values:
    cylinders_dictionary[c] = d

In [12]:
df_limited_years['cylinders'] = df_limited_years.apply( lambda x: x['cylinders'] if pd.isna( x['model'] ) or pd.notna( x['cylinders'] ) else cylinders_dictionary[x['model']] if x['model'] in cylinders_dictionary.keys() else x['cylinders'], axis=1)

In [13]:
df_manufacturer = df_limited_years.loc[df_limited_years['manufacturer'].notna()].copy()
df_manufacturer_model = df_manufacturer.loc[df_manufacturer['model'].notna()].copy()
manufacturer_dictionary = {}
for e, f in df_manufacturer_model[['model', 'manufacturer']].values:
    manufacturer_dictionary[e] = f

In [14]:
df_limited_years['manufacturer'] = df_limited_years.apply( lambda x: x['manufacturer'] if pd.isna( x['model'] ) or pd.notna( x['manufacturer'] ) else manufacturer_dictionary[x['model']] if x['model'] in manufacturer_dictionary.keys() else x['manufacturer'], axis=1)

In [ ]:
df_limited_years.isna().sum()

In [15]:
df_limited_years['cylinders'] = df_limited_years.groupby('model')['cylinders'].ffill()
df_limited_years['manufacturer'] = df_limited_years.groupby('model')['manufacturer'].ffill()
df_limited_years['fuel']=df_limited_years.groupby('model')['fuel'].ffill()
df_limited_years['odometer'] = df_limited_years['odometer'].fillna(df_limited_years.groupby(['year','model'])['odometer'].transform('mean'))
df_limited_years['size']= df_limited_years.groupby('model')['size'].ffill()
df_limited_years['type']= df_limited_years.groupby('model')['type'].ffill()
df_limited_years['manufacturer']= df_limited_years.groupby('model')['manufacturer'].ffill()

In [ ]:
df_limited_years.isna().sum()

In [ ]:
# df_limited_years.drop(['id', 'url', 'region_url', 'VIN', 'paint_color', 'image_url'])

In [ ]:
# import pickle
# with open('car.pkl', 'wb') as carpickle:
#     pickle.dump(df_limited_years,carpickle)

In [ ]:
df_limited_years.isna().sum()

In [ ]:
df_something=df_limited_years.copy()

In [ ]:
df_something.drop(['VIN', 'image_url', 'posting_date','id', 'url','region_url', 'lat','long','paint_color', 'description', 'drive' ], axis=1, inplace=True)

In [ ]:
df_something.info()

In [ ]:
df_something.dropna(inplace=True)
df_something.info()

In [ ]:
# df_ww = df_limited_years.copy()

In [ ]:
# v = df_ww.model.value_counts()
# df_ww['model']=df_ww[df_ww.model.isin(v.index[v.gt(2)])]
# df_ww.isna().sum(), df_ww.info()

In [16]:
unique_manufacturer = df_2['manufacturer'].unique()
manufacturer_list = list(unique_manufacturer)
manufacturer_list =manufacturer_list[1:]
type(manufacturer_list)

list

In [ ]:
manufacturer_list

Maybe keep these to show how the description had the manufacturer's name

In [ ]:
# need to find a way to fill in missing manufacturers. too many Na's to drop

In [ ]:
# see if any other columns have the manufacturer name 
# list is in and return true or false

In [ ]:
# sample = df_2['description'][97]

In [ ]:
# len(sample)

In [ ]:
# pattern = "([a-zA-Z]+(?:'[a-z]+)?)"
# tokenizer = RegexpTokenizer(pattern)
# sample_doc = tokenizer.tokenize(sample)
# sample_doc = [token.lower() for token in sample_doc]
# sw =stopwords.words('english')
# sample_doc = [token for token in sample_doc if token not in sw]

In [ ]:
# sample_doc

In [ ]:
# len(set(sample_doc))

In [ ]:
# print(set(sample_doc))

In [ ]:
# type(sample_doc)

In [ ]:
# # List1
# List1 = unique_manufacturer
 
# # List2
# List2 = sample_doc

# check =  any(item in List1 for item in List2)
 
# if check is True:
#     print("The list {} contains all elements of the list {}".format(List1, List2))    
# else :
#     print("No, List1 doesn't have all elements of the List2.")

In [ ]:
# def list_contains(List1, List2): 
#     check = False
  
#     # Iterate in the 1st list 
#     for m in List1: 
  
#         # Iterate in the 2nd list 
#         for n in List2: 
    
#             # if there is a match
#             if m == n: 
#                 check = True
#                 return check  
                  
#     return check 
      
# # Test Case 1
# List1 = unique_manufacturer
# List2 = sample_doc 
# print("Test Case#1 ", list_contains(List1, List2)) 

In [ ]:
# car = set(unique_manufacturer) & set(sample_doc)
# car

In [ ]:
# df_2['description']=df_2['description'].str.lower()

In [ ]:
# df_2.sample(10)

In [ ]:
# df_2.sample()

In [ ]:
# df_2['exists'] = df.drop('manufacturer', 1).isin(df['manufacturer']).any(1) 
# df_2.exists.value_counts()

In [ ]:
# Only 8 cases where the name of the manufacturer's name is mentioned outside of the manufacturers column therefore wan't
# able to fill in the Na Values

In [17]:
manu_desc=df_limited_years.loc[df_limited_years['manufacturer'].isna(), ['description','manufacturer', 'id']].copy()
manu_desc.head()

,description,manufacturer,id
97,Carvana is the safer way to buy a car During t...,NaN,7309271279
122,"2005 Blue Bird 12 window, 72 passenger bus tha...",NaN,7306198330
135,Carvana is the safer way to buy a car During t...,NaN,7305150004
155,Carvana is the safer way to buy a car During t...,NaN,7302642422
220,2006 STERLING ACTERRA GRAPPLE TRUCK PAC-MAC LO...,NaN,7316423933


In [18]:
manu_desc['description']=manu_desc['description'].str.lower().str.split()

In [21]:
manu_desc.head()

,description,manufacturer,id
97,"[carvana, is, the, safer, way, to, buy, a, car...",NaN,7309271279
122,"[2005, blue, bird, 12, window,, 72, passenger,...",NaN,7306198330
135,"[carvana, is, the, safer, way, to, buy, a, car...",NaN,7305150004
155,"[carvana, is, the, safer, way, to, buy, a, car...",NaN,7302642422
220,"[2006, sterling, acterra, grapple, truck, pac-...",NaN,7316423933


In [19]:
index_nums=[]
m =[]
for index, row in manu_desc.iterrows():
    for manufacturer in manufacturer_list:
        if manufacturer in row['description']:
            index_nums.append(index)
            m.append(manufacturer)            

In [23]:
result = [None]*(len(index_nums)+len(m))
result[::2] = index_nums
result[1::2] =m

for x, y in zip(result[::2], result[1::2]):
    
    print(x,y)


220 gmc
220 ford
601 gmc
696 gmc
696 chevrolet
696 ford
696 nissan
696 dodge
696 mini
787 gmc
787 chevrolet
787 ford
787 ram
787 dodge
794 gmc
794 chevrolet
794 ford
794 ram
794 dodge
798 gmc
798 chevrolet
798 ford
798 ram
798 dodge
815 gmc
815 chevrolet
815 ford
815 ram
815 dodge
822 gmc
822 chevrolet
822 ford
822 ram
822 dodge
824 gmc
824 chevrolet
824 ford
824 ram
824 dodge
832 gmc
832 chevrolet
832 ford
832 ram
832 dodge
834 gmc
834 chevrolet
834 ford
834 ram
834 dodge
836 gmc
836 chevrolet
836 ford
836 ram
836 dodge
838 gmc
838 chevrolet
838 ford
838 ram
838 dodge
848 gmc
848 chevrolet
848 ford
848 ram
848 dodge
849 gmc
849 chevrolet
849 ford
849 ram
849 dodge
854 gmc
854 chevrolet
854 ford
854 ram
854 dodge
1207 chevrolet
1266 gmc
1266 chevrolet
1266 ford
1266 ram
1266 dodge
1267 gmc
1267 chevrolet
1267 ford
1267 ram
1267 dodge
1269 gmc
1269 chevrolet
1269 ford
1269 ram
1269 dodge
1273 gmc
1273 chevrolet
1273 ford
1273 ram
1273 dodge
1274 gmc
1274 chevrolet
1274 ford
1274 ram
127

62505 honda
62505 dodge
62505 lexus
62505 jaguar
62505 volvo
62505 audi
62505 infiniti
62505 lincoln
62505 subaru
62505 acura
62505 hyundai
62505 mercedes-benz
62505 bmw
62505 mitsubishi
62505 volkswagen
62505 porsche
62505 kia
62505 rover
62505 mini
62505 fiat
62505 tesla
62516 gmc
62516 chevrolet
62516 toyota
62516 ford
62516 jeep
62516 nissan
62516 ram
62516 mazda
62516 cadillac
62516 honda
62516 dodge
62516 lexus
62516 jaguar
62516 volvo
62516 audi
62516 infiniti
62516 lincoln
62516 subaru
62516 acura
62516 hyundai
62516 mercedes-benz
62516 bmw
62516 mitsubishi
62516 volkswagen
62516 porsche
62516 kia
62516 rover
62516 mini
62516 fiat
62516 tesla
62526 gmc
62526 chevrolet
62526 toyota
62526 ford
62526 jeep
62526 nissan
62526 ram
62526 mazda
62526 cadillac
62526 honda
62526 dodge
62526 lexus
62526 jaguar
62526 volvo
62526 audi
62526 infiniti
62526 lincoln
62526 subaru
62526 acura
62526 hyundai
62526 mercedes-benz
62526 bmw
62526 mitsubishi
62526 volkswagen
62526 porsche
62526 kia
62

111752 ford
111884 mercedes-benz
112089 mitsubishi
112262 chevrolet
112278 chevrolet
112363 bmw
112539 chevrolet
112539 toyota
112539 ford
112539 honda
112539 dodge
112539 chrysler
112539 volvo
112539 audi
112539 acura
112539 porsche
112541 chevrolet
112541 toyota
112541 ford
112541 honda
112541 dodge
112541 chrysler
112541 volvo
112541 audi
112541 acura
112541 porsche
112603 morgan
112633 toyota
112754 ford
112754 ram
113558 mazda
113925 hyundai
114035 hyundai
114275 chevrolet
114442 bmw
114472 chevrolet
114495 ford
114615 honda
114765 toyota
114779 ford
115072 ford
115130 toyota
116562 chevrolet
116562 ford
116600 mitsubishi
116611 jeep
116754 gmc
116754 ford
116835 jaguar
116938 gmc
116938 toyota
116938 ford
116938 nissan
116938 ram
116938 cadillac
116938 honda
116938 dodge
116938 lexus
116938 jaguar
116938 audi
116938 infiniti
116938 mercedes-benz
116938 bmw
116938 porsche
116938 tesla
116938 ferrari
116939 gmc
116939 toyota
116939 ford
116939 nissan
116939 ram
116939 cadillac
1169

186357 mazda
186578 jeep
186809 lincoln
186811 gmc
186857 mazda
187018 subaru
187067 toyota
187067 nissan
187067 honda
187067 subaru
187067 volkswagen
187067 kia
187178 nissan
187184 volkswagen
187188 mazda
187193 lexus
187196 honda
187197 toyota
187197 nissan
187197 mazda
187197 honda
187197 volvo
187197 audi
187197 subaru
187197 hyundai
187197 mercedes-benz
187197 volkswagen
187197 kia
187197 mini
187229 nissan
187229 mitsubishi
187231 morgan
187248 lincoln
187517 gmc
187517 chevrolet
187517 toyota
187517 honda
187517 dodge
187517 audi
187517 infiniti
187517 hyundai
187589 hyundai
188012 gmc
188012 chevrolet
188012 ford
188012 ram
188012 dodge
188012 mitsubishi
188012 morgan
188460 hyundai
188728 toyota
189085 honda
189152 gmc
189152 ford
189152 jeep
189152 nissan
189152 mazda
189152 honda
189152 hyundai
189152 kia
189857 chrysler
189860 toyota
189860 dodge
189860 chrysler
190059 honda
190570 gmc
190570 chevrolet
190570 morgan
190571 gmc
190571 chevrolet
190571 morgan
190572 gmc
1905

251879 chevrolet
251879 ford
251879 ram
251879 dodge
251880 gmc
251880 chevrolet
251880 ford
251880 ram
251880 dodge
251893 toyota
251893 ford
251893 nissan
251893 mazda
251893 honda
251893 dodge
251893 chrysler
251893 audi
251893 acura
251893 bmw
251893 kia
252319 gmc
252319 chevrolet
252319 ford
252319 jeep
252319 cadillac
252319 dodge
252319 lexus
252319 jaguar
252319 buick
252319 chrysler
252319 volvo
252319 audi
252319 infiniti
252319 lincoln
252319 acura
252319 bmw
252319 volkswagen
252319 porsche
252319 rover
252572 nissan
252626 dodge
252728 ferrari
253147 volvo
253149 chevrolet
253284 gmc
253284 chevrolet
253284 ford
253284 jeep
253284 cadillac
253284 dodge
253284 lexus
253284 jaguar
253284 buick
253284 chrysler
253284 volvo
253284 audi
253284 infiniti
253284 lincoln
253284 acura
253284 bmw
253284 volkswagen
253284 porsche
253284 rover
254150 nissan
254223 mitsubishi
255606 volkswagen
257150 gmc
257150 ford
257150 ram
257150 dodge
257152 gmc
257152 ford
257152 ram
257152 dodge

370352 infiniti
370353 infiniti
370354 infiniti
370355 infiniti
370588 morgan
370605 chevrolet
371248 gmc
371248 chevrolet
371248 ford
371248 chrysler
371597 volvo
371642 lexus
371806 volvo
372014 toyota
372014 nissan
372014 jaguar
372014 infiniti
372014 volkswagen
372014 tesla
372256 volvo
372258 ford
372258 dodge
372258 volvo
372388 volvo
372652 chevrolet
372652 ford
372652 ram
372652 dodge
372766 gmc
372766 chevrolet
372766 ford
372766 ram
372766 dodge
372930 toyota
372930 nissan
372930 jaguar
372930 infiniti
372930 volkswagen
372930 tesla
372934 volvo
373164 toyota
373164 nissan
373164 jaguar
373164 infiniti
373164 volkswagen
373164 tesla
373299 volvo
373427 volvo
374015 gmc
374015 ford
374445 chevrolet
374445 ford
374445 jeep
374445 mitsubishi
374695 hyundai
374766 gmc
374766 ford
374769 volvo
374925 chevrolet
374925 ram
374925 tesla
375085 dodge
375088 chevrolet
375088 hyundai
375088 bmw
375426 cadillac
375426 hyundai
375530 ford
375728 mini
375921 subaru
376107 gmc
376107 chevro

In [20]:
manu_desc_completed = pd.DataFrame(index_nums, columns=['index'])
manu_desc_completed['manufacturer'] = m

In [25]:
manu_desc_completed.head(15)

,index,manufacturer
0,220,gmc
1,220,ford
2,601,gmc
3,696,gmc
4,696,chevrolet
5,696,ford
6,696,nissan
7,696,dodge
8,696,mini
9,787,gmc


In [21]:
manu_desc_completed_new=manu_desc_completed.drop_duplicates(subset=['index'])
manu_desc_completed_new.manufacturer.value_counts()

gmc              869
ford             297
toyota           228
chevrolet        185
dodge            105
volvo             95
hyundai           84
honda             83
nissan            78
chrysler          65
jeep              62
subaru            49
ram               44
infiniti          42
bmw               42
mercedes-benz     41
volkswagen        41
mini              39
mazda             33
mitsubishi        30
cadillac          27
morgan            26
lexus             24
ferrari           19
lincoln           16
audi              16
acura             15
pontiac           15
kia               13
mercury            5
jaguar             4
saturn             4
porsche            4
rover              4
fiat               4
buick              4
tesla              4
Name: manufacturer, dtype: int64

In [ ]:
manu_desc_completed_new.head(15)

In [22]:
manu_desc_completed_new.set_index('index',inplace=True )
manu_desc_completed_new.head()

,manufacturer
index,
220,gmc
601,gmc
696,gmc
787,gmc
794,gmc


In [ ]:
len(manu_desc_completed_new.manufacturer)

In [23]:
df_yo= df_limited_years.copy()

In [24]:
df_yo.update(manu_desc_completed_new)
df_yo.isna().sum()

id                   0
url                  0
region               0
region_url           0
price                0
year                 0
manufacturer      7073
model             2530
condition        17764
cylinders        33444
fuel              1410
odometer           392
title_status      7004
transmission      1645
VIN             120074
drive           105630
size             91605
type              9966
paint_color     101144
image_url            0
description          2
state                0
lat               3283
long              3283
posting_date         0
dtype: int64

In [ ]:
df_yo.iloc[170:180]

In [25]:
unique_condition = df_2['condition'].unique()
condition_list = list(unique_condition)
condition_list =condition_list[1:]
condition_list
# type(condition_list)

['good', 'excellent', 'fair', 'like new', 'new', 'salvage']

In [26]:
cond_desc=df_limited_years.loc[df_limited_years['condition'].isna(), ['description','condition', 'id']].copy()
cond_desc.head()

,description,condition,id
195,2012 Black 140-inch Dodge Charger Limousine #1...,NaN,7316711193
225,2011 Chevrolet Silverado 1500 LTZ Crew Cab 4WD...,NaN,7316396026
226,2015 Chevrolet Silverado 1500 LT Crew Cab 4WD ...,NaN,7316395026
227,2014 Chevrolet Silverado 1500 2LT Crew Cab Lon...,NaN,7316393935
265,"2006 *Suzuki* *XL-7* SUV - $2,975Call Us Toda...",NaN,7315970659


In [27]:
cond_desc['description']=cond_desc['description'].str.lower().str.split()

In [28]:
index_nums2=[]
c =[]
for index, row in cond_desc.iterrows():
    for condition in condition_list:
        if condition in row['description']:
            index_nums2.append(index)
            c.append(condition)  
result2 = [None]*(len(index_nums2)+len(c))
result2[::2] = index_nums2
result2[1::2] =c

for x, y in zip(result2[::2], result2[1::2]):
    
    print(x,y)

414 new
460 new
508 new
696 new
904 good
905 good
905 new
907 good
907 new
908 new
1041 new
1319 new
1756 new
1818 good
1818 new
1821 good
1821 new
1823 good
1823 new
1826 good
1826 new
1834 good
1834 new
1857 good
1857 new
1864 good
1864 new
1891 good
1891 new
1914 good
1914 new
1928 good
1928 new
1930 good
1930 new
1939 good
1939 new
1950 good
1950 new
1951 good
1951 new
1960 good
1960 new
1961 good
1961 new
1975 good
1975 new
1981 good
1981 new
1987 good
1987 new
1989 good
1989 new
2011 good
2011 new
2073 good
2073 new
2078 good
2078 new
2083 good
2083 new
2099 good
2527 good
2593 new
2847 new
2896 new
3017 new
3123 new
3212 new
3453 new
4915 new
5004 excellent
5005 new
5132 new
5181 new
5182 excellent
5183 good
5183 fair
5208 good
5342 good
5464 new
5498 excellent
5498 new
5638 fair
5639 good
5640 good
5640 excellent
5749 good
5751 new
5759 good
5759 new
5820 excellent
5821 new
5965 good
6077 new
6095 good
6095 new
6229 good
6350 new
6488 new
6584 new
6614 new
6616 good
6616 excell

23031 new
23034 new
23051 good
23053 good
23054 good
23055 good
23056 good
23057 good
23058 good
23078 new
23086 excellent
23086 new
23139 good
23139 fair
23139 new
23166 good
23167 good
23171 good
23179 good
23222 good
23222 fair
23222 new
23224 good
23224 fair
23224 new
23234 new
23235 good
23235 new
23238 good
23239 good
23240 good
23241 good
23242 good
23304 good
23304 fair
23304 new
23305 good
23305 fair
23305 new
23306 good
23306 fair
23306 new
23408 new
23412 excellent
23412 new
23424 good
23424 fair
23424 new
23491 new
23526 good
23527 good
23527 new
23528 good
23528 new
23529 good
23529 new
23530 good
23531 good
23533 good
23534 good
23534 new
23535 good
23536 good
23577 good
23577 fair
23577 new
23580 good
23580 fair
23580 new
23654 new
23655 new
23656 new
23677 new
23918 good
23931 new
24002 new
24009 good
24009 new
24031 new
24070 new
24071 new
24178 excellent
24372 new
24378 new
24421 new
24424 new
24465 new
24466 new
24468 new
24483 new
24512 new
24628 new
24631 new
24686

69138 fair
69138 new
69139 good
69139 fair
69139 new
69140 good
69140 fair
69140 new
69142 good
69142 fair
69142 new
69211 good
69211 fair
69211 new
69215 good
69215 fair
69215 new
69216 good
69216 fair
69216 new
69357 good
69357 fair
69357 new
69360 good
69360 fair
69360 new
69361 good
69361 fair
69361 new
69363 good
69363 fair
69363 new
69367 good
69367 fair
69367 new
69565 good
69565 fair
69565 new
69567 good
69567 fair
69567 new
69595 good
69595 fair
69595 new
69640 good
69642 good
69670 good
69670 fair
69670 new
69671 good
69671 fair
69671 new
69673 good
69673 fair
69673 new
69677 good
69677 fair
69677 new
69678 good
69678 fair
69678 new
69679 good
69679 fair
69679 new
69680 good
69680 fair
69680 new
69681 good
69681 fair
69681 new
69682 good
69682 fair
69682 new
69684 good
69684 fair
69684 new
69685 good
69685 fair
69685 new
69686 good
69686 fair
69686 new
69687 good
69687 fair
69687 new
69690 good
69690 fair
69690 new
69696 good
69696 fair
69696 new
69736 good
69736 fair
69736 n

98023 good
98023 new
98032 good
98032 new
98074 good
98074 new
98107 new
98195 excellent
98195 new
98252 good
98252 new
98276 excellent
98331 good
98481 good
98481 new
98516 good
98539 new
98610 good
98655 good
98660 good
98660 new
98679 new
98743 good
98743 new
98756 good
98811 good
98818 new
98819 new
98939 new
98941 new
98956 new
98973 new
98979 new
98981 new
99008 good
99097 good
99165 excellent
99183 good
99183 new
99193 excellent
99193 new
99224 excellent
99256 good
99256 new
99277 new
99320 good
99420 good
99420 new
99435 good
99435 excellent
99507 new
99512 good
99517 good
99517 excellent
99517 new
99550 good
99555 new
99561 new
99632 new
99634 good
99635 good
99639 excellent
99642 excellent
99643 good
99646 good
99704 good
99704 new
99775 new
99779 new
99782 good
99824 good
99904 new
99906 new
99932 excellent
100040 good
100069 good
100069 new
100091 excellent
100091 new
100097 good
100097 new
100146 excellent
100152 good
100161 good
100161 new
100187 good
100187 excellent
100

157171 good
157359 salvage
157454 new
157570 new
157570 salvage
157788 salvage
157789 salvage
157808 salvage
157832 new
157837 excellent
157837 new
157838 new
157841 good
157841 new
157962 excellent
157962 new
157963 new
157967 new
158117 salvage
158155 new
158155 salvage
158163 new
158163 salvage
158406 new
158420 good
158420 new
158499 excellent
158499 new
158720 excellent
158721 good
158822 new
158877 new
158895 new
158934 new
159106 salvage
159193 good
159193 excellent
159193 new
159204 good
159204 new
159215 new
159234 new
159356 good
159399 good
159465 good
160510 excellent
160575 good
160575 new
160681 excellent
161487 salvage
161975 fair
161984 salvage
161995 new
161995 salvage
162255 good
162255 salvage
162257 good
162258 excellent
162258 new
162312 excellent
162469 salvage
162470 salvage
162482 salvage
162619 new
162786 new
162819 excellent
163011 salvage
163087 good
163087 new
163177 salvage
163241 new
163241 salvage
164176 good
164176 fair
164176 new
164336 good
164336 exce

229711 new
229749 good
229809 excellent
229809 new
229816 new
229852 good
229853 new
229855 new
229914 good
229943 good
229943 new
230038 good
230038 new
230045 good
230057 good
230057 new
230083 excellent
230197 new
230205 good
230316 new
230317 new
230504 new
230506 new
230517 new
230634 good
230635 new
230662 excellent
230673 good
230727 good
230756 new
230759 new
230840 new
230917 good
230924 new
230925 good
230983 good
231026 good
231026 new
231037 new
231048 good
231048 new
231085 good
231085 new
231097 excellent
231142 good
231142 new
231195 new
231200 good
231517 new
231536 new
231613 new
231689 good
231694 new
231805 salvage
231872 good
231872 new
231933 good
231933 new
232119 good
232193 good
232193 new
232225 excellent
232314 new
232336 good
232336 excellent
232863 new
232865 good
232868 new
232877 new
232892 new
232942 new
233015 good
233281 new
233467 new
233539 new
233560 new
233815 good
233815 new
233879 excellent
233932 new
233939 new
234071 excellent
234192 good
235105

260360 good
260360 new
260518 good
260518 fair
260518 new
260523 good
260523 fair
260523 new
260526 good
260526 fair
260526 new
260527 good
260527 fair
260527 new
260529 good
260529 fair
260529 new
260537 good
260537 fair
260537 new
260619 good
260859 new
261095 salvage
261218 excellent
261223 excellent
261225 good
261225 excellent
261225 new
261226 good
261226 excellent
261226 new
261309 new
261315 good
261372 good
261455 good
261455 fair
261455 new
261458 good
261458 fair
261458 new
261464 good
261464 fair
261464 new
261552 new
261596 excellent
261596 new
261609 new
261677 good
261678 good
261767 excellent
261767 new
261770 new
261787 good
261787 new
261846 good
261846 fair
261846 new
261848 good
261848 fair
261848 new
261872 new
261888 salvage
261909 good
261978 good
261978 fair
261978 new
261981 new
261997 excellent
261997 new
262000 new
262060 good
262060 fair
262060 new
262061 good
262061 fair
262061 new
262063 good
262063 fair
262063 new
262064 good
262064 fair
262064 new
262067

323602 new
323603 good
323604 good
323606 good
323610 good
323611 good
323612 good
323613 good
323613 new
323614 good
323614 new
323617 good
323619 good
323619 new
323620 good
323622 good
323622 new
323622 salvage
323623 good
323628 good
323628 new
323630 good
323630 new
323632 good
323633 good
323633 new
323634 good
323636 good
323638 good
323638 new
323639 good
323641 good
323642 good
323647 good
323650 excellent
323651 excellent
323652 excellent
323666 excellent
323667 excellent
323673 excellent
323673 new
323674 excellent
323674 new
323675 excellent
323801 new
323804 new
323806 new
323812 excellent
323876 excellent
323876 new
323929 good
323948 new
323988 good
324000 good
324006 good
324050 good
324094 new
324097 new
324131 excellent
324189 good
324210 new
324213 new
324214 excellent
324214 new
324224 new
324225 excellent
324225 new
324248 excellent
324309 new
324347 new
324355 new
324366 new
324412 fair
324421 new
324433 good
324433 excellent
324433 new
324442 excellent
324442 new

375795 fair
375795 new
375985 good
375985 excellent
375985 new
376097 salvage
376105 salvage
376106 salvage
376107 salvage
376762 new
376833 good
376854 good
376869 new
376872 good
376872 fair
376872 new
376906 new
376978 good
376978 new
376980 new
377051 good
377051 fair
377051 new
377158 excellent
377158 new
377159 new
377162 good
377162 new
377171 good
377171 fair
377171 new
377192 excellent
377192 new
377193 new
377261 good
377261 fair
377261 new
377281 new
377283 excellent
377283 new
377287 good
377287 new
377308 excellent
377308 new
377363 good
377363 fair
377363 new
377402 new
377482 good
377482 fair
377482 new
377495 new
377500 good
377500 new
377528 new
377547 new
377577 good
377577 new
377579 new
377668 excellent
377668 new
377669 new
377671 good
377671 new
377689 excellent
377689 new
377690 new
377754 new
377758 good
377758 new
377766 excellent
377766 new
377828 new
377857 good
377857 new
377865 new
377886 new
377986 good
377986 new
377988 new
378122 excellent
378122 new
378

In [29]:
cond_desc_completed = pd.DataFrame(index_nums2, columns=['index'])
cond_desc_completed['condition'] = c
cond_desc_completed_new=cond_desc_completed.drop_duplicates(subset=['index'])
# cond_desc_completed_new.condition.value_counts()
cond_desc_completed_new.set_index('index',inplace=True )
cond_desc_completed_new.head()

,condition
index,
414,new
460,new
508,new
696,new
904,good


In [30]:
df_yo.update(cond_desc_completed_new)
df_yo.isna().sum()

id                   0
url                  0
region               0
region_url           0
price                0
year                 0
manufacturer      7073
model             2530
condition         9894
cylinders        33444
fuel              1410
odometer           392
title_status      7004
transmission      1645
VIN             120074
drive           105630
size             91605
type              9966
paint_color     101144
image_url            0
description          2
state                0
lat               3283
long              3283
posting_date         0
dtype: int64

In [ ]:
# unique_cylinders = df_2['cylinders'].unique()
# cylinders_list = list(unique_cylinders)
# cylinders_list = cylinders_list[1:]
# cylinders_list

In [ ]:
# cyld_desc=df_limited_years.loc[df_limited_years['cylinders'].isna(), ['description','cylinders', 'id']].copy()
# cyld_desc.head()

In [ ]:
# cyld_desc['description']=cyld_desc['description'].str.lower().str.split()

In [ ]:

# ll=[]
# cy =[]
# for index, row in cyld_desc.iterrows():
#     for cylinders in cylinders_list:
#         if cylinders in row['description']:
#             ll.append(index)
#             cy.append(manufacturer)  
# result3 = [None]*(len(ll)+len(cy))
# result3[::2] = ll
# result3[1::2] =cy
# result3
# # for x, y in zip(result3[::2], result3[1::2]):
    
# #     print(x,y)

In [31]:
unique_fuel = df_limited_years['fuel'].unique()
fuel_list = list(unique_fuel)
fuel_list = ['gas', 'diesel', 'hybrid', 'electric']

In [32]:
fuel_desc=df_limited_years.loc[df_limited_years['fuel'].isna(), ['description','fuel', 'id']].copy()
fuel_desc['description']=fuel_desc['description'].str.lower().str.split()
fuel_desc.head()

,description,fuel,id
225,"[2011, chevrolet, silverado, 1500, ltz, crew, ...",NaN,7316396026
226,"[2015, chevrolet, silverado, 1500, lt, crew, c...",NaN,7316395026
227,"[2014, chevrolet, silverado, 1500, 2lt, crew, ...",NaN,7316393935
314,"[2011, *jeep*, *grand, cherokee*, rwd, 4dr, la...",NaN,7315518138
368,"[2010, *chevrolet*, *hhr*, fwd, 4dr, lt, w/1lt...",NaN,7314945978


In [33]:
index_nums4=[]
f =[]
for index, row in fuel_desc.iterrows():
    for fuel in fuel_list:
        if fuel in row['description']:
            index_nums4.append(index)
            f.append(fuel)  
result4 = [None]*(len(index_nums4)+len(f))
result4[::2] = index_nums4
result4[1::2] =f

for x, y in zip(result4[::2], result4[1::2]):
    
    print(x,y)

1569 hybrid
2278 hybrid
3621 hybrid
4952 hybrid
10399 electric
10472 diesel
10475 diesel
11597 electric
11675 electric
12374 electric
12465 diesel
12465 electric
17017 electric
17112 electric
28564 hybrid
28567 hybrid
28768 hybrid
28826 hybrid
38745 electric
40446 electric
55256 electric
64111 gas
64111 hybrid
81736 hybrid
81885 diesel
81890 hybrid
81929 diesel
81930 electric
81944 diesel
81948 diesel
81949 gas
81953 hybrid
81956 gas
82205 gas
82209 gas
82211 gas
82216 hybrid
82247 hybrid
82258 gas
82290 gas
82294 hybrid
82297 gas
82324 gas
82327 diesel
82332 gas
82377 gas
82379 gas
82380 gas
82383 diesel
82454 hybrid
82455 gas
82502 gas
82502 hybrid
82504 diesel
82522 gas
82526 hybrid
82556 gas
82692 hybrid
82761 hybrid
82790 gas
82836 hybrid
87859 diesel
88188 electric
88189 hybrid
88189 electric
89194 electric
89195 hybrid
89259 electric
89268 electric
89320 electric
89322 diesel
89322 electric
90106 electric
90153 electric
90746 hybrid
90746 electric
90765 electric
90778 electric
9

In [34]:
fuel_desc_completed = pd.DataFrame(index_nums4, columns=['index'])
fuel_desc_completed['fuel'] = f
fuel_desc_completed_new=fuel_desc_completed.drop_duplicates(subset=['index'])
fuel_desc_completed_new.set_index('index',inplace=True )
fuel_desc_completed_new.head()

,fuel
index,
1569,hybrid
2278,hybrid
3621,hybrid
4952,hybrid
10399,electric


In [35]:
df_yo.update(fuel_desc_completed_new)
df_yo.isna().sum()

id                   0
url                  0
region               0
region_url           0
price                0
year                 0
manufacturer      7073
model             2530
condition         9894
cylinders        33444
fuel              1215
odometer           392
title_status      7004
transmission      1645
VIN             120074
drive           105630
size             91605
type              9966
paint_color     101144
image_url            0
description          2
state                0
lat               3283
long              3283
posting_date         0
dtype: int64

In [36]:
unique_transmission = df_limited_years['transmission'].unique()
trasnmission_list = list(unique_transmission)
transmission_list = ['automatic', 'manual']

In [37]:
tran_desc=df_limited_years.loc[df_limited_years['transmission'].isna(), ['description','transmission', 'id']].copy()
tran_desc['description']=tran_desc['description'].str.lower().str.split()
tran_desc.head()

,description,transmission,id
2961,"[2016, nissan, murano, sv, offered, by:, autov...",NaN,7312867213
5088,"[2012, ford, f-550, super, duty, 4x4, 2dr, reg...",NaN,7316461977
5250,"[bad, credit, -, no, credit..., we, can, help!...",NaN,7315616467
5284,"[2006, ford, f-550, super, duty, 4x4, 4dr, sup...",NaN,7315552489
5384,"[this, chevrolet, corvette, can, be, yours, to...",NaN,7315160850


In [38]:
index_nums5=[]
t =[]
for index, row in tran_desc.iterrows():
    for transmission in transmission_list:
        if transmission in row['description']:
            index_nums5.append(index)
            t.append(transmission)  
result5 = [None]*(len(index_nums5)+len(t))
result5[::2] = index_nums5
result5[1::2] =t

for x, y in zip(result5[::2], result5[1::2]):
    
    print(x,y)

8793 automatic
8800 automatic
8807 automatic
8808 automatic
8810 automatic
8819 automatic
14704 automatic
30669 automatic
30671 automatic
30833 automatic
30980 manual
51372 automatic
52183 automatic
57441 automatic
57442 automatic
57443 automatic
57444 automatic
57445 automatic
57446 automatic
57447 automatic
57448 automatic
57449 automatic
57450 automatic
57789 automatic
57790 automatic
57792 automatic
57794 automatic
57795 automatic
57796 automatic
57797 automatic
57798 automatic
57800 automatic
57802 automatic
57803 automatic
57804 manual
57805 automatic
57806 automatic
57810 automatic
58405 automatic
58406 automatic
58407 manual
58408 automatic
58410 automatic
58418 automatic
58419 automatic
58420 automatic
58421 automatic
58422 automatic
58423 automatic
58424 automatic
58425 automatic
58426 automatic
58428 automatic
58429 automatic
58430 automatic
61154 automatic
62945 automatic
76107 automatic
76214 automatic
76225 automatic
76334 automatic
77196 manual
77266 automatic
77266 manu

In [39]:
tran_desc_completed = pd.DataFrame(index_nums5, columns=['index'])
tran_desc_completed['transmission'] = t
tran_desc_completed_new=tran_desc_completed.drop_duplicates(subset=['index'])
tran_desc_completed_new.set_index('index',inplace=True )
tran_desc_completed_new.head()

,transmission
index,
8793,automatic
8800,automatic
8807,automatic
8808,automatic
8810,automatic


In [40]:
df_yo.update(tran_desc_completed_new)
df_yo.isna().sum()

id                   0
url                  0
region               0
region_url           0
price                0
year                 0
manufacturer      7073
model             2530
condition         9894
cylinders        33444
fuel              1215
odometer           392
title_status      7004
transmission      1158
VIN             120074
drive           105630
size             91605
type              9966
paint_color     101144
image_url            0
description          2
state                0
lat               3283
long              3283
posting_date         0
dtype: int64

In [41]:
unique_drive = df_limited_years['drive'].unique()
drive_list = list(unique_drive)
drive_list= drive_list[1:]
drive_list

['rwd', '4wd', 'fwd']

In [42]:
driv_desc=df_limited_years.loc[df_limited_years['drive'].isna(), ['description','drive', 'id']].copy()
driv_desc['description']=driv_desc['description'].str.lower().str.split()
driv_desc.head()

,description,drive,id
27,"[carvana, is, the, safer, way, to, buy, a, car...",NaN,7316814884
28,"[carvana, is, the, safer, way, to, buy, a, car...",NaN,7316814758
29,"[carvana, is, the, safer, way, to, buy, a, car...",NaN,7316814989
30,"[carvana, is, the, safer, way, to, buy, a, car...",NaN,7316743432
39,"[carvana, is, the, safer, way, to, buy, a, car...",NaN,7315770394


In [ ]:
driv_desc.isna().sum()

In [43]:
driv_desc.dropna(subset=['description'], axis=0, inplace=True)

In [44]:
index_nums6=[]
d =[]
for index, row in driv_desc.iterrows():
    for drive in drive_list:
        if drive in row['description']:
            index_nums6.append(index)
            d.append(drive)  
result6 = [None]*(len(index_nums6)+len(d))
result6[::2] = index_nums6
result6[1::2] =d

for x, y in zip(result6[::2], result6[1::2]):
    
    print(x,y)

423 4wd
504 4wd
510 4wd
659 4wd
1038 4wd
1042 4wd
1357 rwd
1489 4wd
1676 4wd
1818 4wd
1821 4wd
1823 fwd
1826 4wd
1831 4wd
1834 fwd
1835 4wd
1864 fwd
1869 4wd
1871 4wd
1872 4wd
1882 fwd
1887 4wd
1891 4wd
1892 fwd
1899 fwd
1900 fwd
1913 4wd
1914 4wd
1915 fwd
1918 fwd
1928 4wd
1930 rwd
1933 4wd
1937 4wd
1939 fwd
1947 4wd
1950 4wd
1951 4wd
1953 rwd
1960 4wd
1961 4wd
1962 fwd
1964 rwd
1971 4wd
1972 fwd
1975 4wd
1978 fwd
1981 fwd
1982 4wd
1987 4wd
1989 4wd
1996 fwd
1997 4wd
2000 fwd
2006 fwd
2007 4wd
2011 fwd
2012 fwd
2073 4wd
2077 fwd
2078 fwd
2082 fwd
2083 4wd
2125 4wd
2128 fwd
2590 4wd
2594 4wd
2611 4wd
2722 fwd
2731 rwd
2791 4wd
3010 4wd
3098 4wd
3170 fwd
3172 rwd
3179 rwd
3209 4wd
3213 4wd
3436 4wd
3583 4wd
3646 fwd
3675 rwd
3676 fwd
3734 fwd
3745 fwd
3871 4wd
5114 fwd
5168 fwd
5457 fwd
5554 fwd
5816 4wd
5829 4wd
6046 4wd
6756 4wd
7116 4wd
7208 4wd
7371 4wd
8468 fwd
8469 fwd
8470 fwd
8473 4wd
8475 4wd
8480 fwd
8481 fwd
8482 fwd
8492 4wd
8530 fwd
8534 fwd
8535 fwd
8536 fwd
8537 fwd
8538 

50000 fwd
50007 fwd
50038 fwd
50073 fwd
50084 fwd
50102 fwd
50144 4wd
50189 fwd
50249 4wd
50250 4wd
50254 rwd
50259 4wd
50297 fwd
50326 4wd
50327 4wd
50329 4wd
50331 4wd
50332 4wd
50434 fwd
50443 fwd
50608 fwd
50620 rwd
50621 4wd
50623 4wd
50625 4wd
50628 fwd
50634 4wd
50648 fwd
50670 fwd
50674 fwd
50706 fwd
50720 fwd
50721 fwd
50724 fwd
50817 fwd
50826 fwd
50877 fwd
50927 fwd
51040 4wd
51047 fwd
51050 fwd
51074 fwd
51091 fwd
51098 fwd
51120 fwd
51128 fwd
51187 rwd
51190 4wd
51192 rwd
51208 fwd
51211 fwd
51246 fwd
51261 fwd
51297 4wd
51298 4wd
51302 4wd
51372 rwd
51389 fwd
51393 rwd
51398 4wd
51399 4wd
51498 fwd
51519 fwd
51522 4wd
51537 4wd
51542 4wd
51544 rwd
51546 fwd
51555 4wd
51571 4wd
51653 rwd
51929 rwd
51960 4wd
52027 4wd
52095 4wd
52117 rwd
52183 rwd
52272 4wd
52297 fwd
52301 4wd
52316 rwd
52317 4wd
52582 4wd
52592 rwd
52726 rwd
52819 fwd
52821 fwd
52822 rwd
52823 rwd
52824 fwd
52825 fwd
52826 fwd
52828 4wd
52923 4wd
52940 4wd
52967 4wd
53001 fwd
53052 4wd
53066 fwd
53092 fwd


104468 fwd
104513 fwd
104580 fwd
104656 fwd
104663 fwd
104679 fwd
104702 rwd
104710 fwd
104744 fwd
104769 fwd
104807 rwd
104849 fwd
104864 fwd
104869 fwd
104884 fwd
104934 4wd
104936 4wd
104950 4wd
104971 rwd
105106 4wd
105132 4wd
105148 fwd
105162 fwd
105184 4wd
105201 fwd
105206 rwd
105207 4wd
105211 4wd
105214 4wd
105215 fwd
105216 4wd
105217 rwd
105244 rwd
105266 4wd
105278 4wd
105283 4wd
105502 fwd
105556 fwd
105594 4wd
105599 fwd
105623 fwd
105637 4wd
105644 fwd
105798 4wd
105799 rwd
105800 fwd
105811 fwd
105819 rwd
105820 fwd
105823 fwd
105824 fwd
105827 fwd
105829 fwd
105860 fwd
105873 4wd
105904 4wd
105929 rwd
105957 4wd
105987 fwd
106008 rwd
106009 rwd
106013 rwd
106035 4wd
106128 4wd
106131 4wd
106158 4wd
106222 fwd
106244 fwd
106250 fwd
106256 fwd
106347 fwd
106362 4wd
106386 fwd
106388 fwd
106390 rwd
106391 rwd
106392 rwd
106393 rwd
106404 fwd
106418 fwd
106480 4wd
106497 rwd
106504 4wd
106550 rwd
106578 fwd
106626 4wd
106633 fwd
106640 fwd
106649 rwd
106656 fwd
106664 fwd

161459 fwd
161460 fwd
161461 4wd
161461 fwd
161462 fwd
161464 fwd
161465 fwd
161466 fwd
161467 fwd
161468 fwd
161469 fwd
161470 fwd
161471 fwd
161472 4wd
161472 fwd
161473 fwd
161475 4wd
161475 fwd
161476 fwd
161477 fwd
161478 4wd
161481 rwd
161517 4wd
161575 fwd
161598 fwd
161639 fwd
161640 fwd
161641 fwd
161642 fwd
161643 fwd
161644 4wd
161644 fwd
161645 fwd
161646 fwd
161647 fwd
161648 fwd
161649 fwd
161650 fwd
161651 fwd
161652 fwd
161653 fwd
161655 fwd
161656 fwd
161658 fwd
161659 4wd
161659 fwd
161660 fwd
161661 fwd
161662 fwd
161663 fwd
161664 fwd
161665 fwd
161666 fwd
161667 fwd
161668 fwd
161669 fwd
161670 fwd
161671 fwd
161672 fwd
161673 fwd
161674 fwd
161675 fwd
161676 fwd
161677 fwd
161678 fwd
161679 fwd
161705 fwd
161708 4wd
161728 4wd
161732 4wd
161732 fwd
161733 fwd
161734 fwd
161735 4wd
161735 fwd
161737 4wd
161737 fwd
161738 4wd
161738 fwd
161739 fwd
161741 fwd
161742 fwd
161743 fwd
161744 fwd
161746 fwd
161747 fwd
161749 fwd
161750 fwd
161751 fwd
161753 fwd
161754 fwd

195132 fwd
195133 4wd
195134 fwd
195135 rwd
195143 4wd
195144 fwd
195145 fwd
195147 4wd
195148 fwd
195149 4wd
195152 fwd
195160 fwd
195164 4wd
195179 fwd
195181 4wd
195182 4wd
195183 4wd
195184 fwd
195185 fwd
195186 4wd
195193 fwd
195194 fwd
195207 fwd
195208 fwd
195209 fwd
195211 fwd
195212 fwd
195213 4wd
195214 4wd
195215 fwd
195245 4wd
195261 4wd
195262 rwd
195264 fwd
195265 fwd
195266 fwd
195274 4wd
195275 fwd
195276 fwd
195277 4wd
195278 fwd
195279 fwd
195280 fwd
195281 rwd
195282 fwd
195283 4wd
195297 4wd
195298 fwd
195299 4wd
195302 fwd
195303 fwd
195304 4wd
195305 fwd
195306 fwd
195307 4wd
195308 fwd
195314 fwd
195317 fwd
195322 fwd
195339 4wd
195341 4wd
195342 4wd
195344 fwd
195387 fwd
195388 fwd
195389 fwd
195390 4wd
195410 fwd
195412 4wd
195415 rwd
195416 4wd
195426 fwd
195427 4wd
195428 4wd
195429 4wd
195430 4wd
195442 4wd
195445 fwd
195448 fwd
195449 4wd
195459 fwd
195461 fwd
195462 4wd
195463 fwd
195480 4wd
195481 fwd
195482 rwd
195483 4wd
195484 fwd
195492 4wd
195493 4wd

207234 4wd
207235 fwd
207237 fwd
207238 4wd
207242 fwd
207254 fwd
207255 fwd
207256 fwd
207257 4wd
207258 fwd
207310 rwd
207312 fwd
207313 4wd
207314 4wd
207317 fwd
207321 fwd
207335 fwd
207337 fwd
207339 4wd
207354 fwd
207355 fwd
207356 fwd
207361 4wd
207378 rwd
207380 fwd
207381 4wd
207382 fwd
207385 4wd
207391 fwd
207392 4wd
207393 fwd
207394 fwd
207395 fwd
207396 fwd
207399 4wd
207400 4wd
207401 fwd
207403 4wd
207404 4wd
207416 4wd
207426 fwd
207433 4wd
207437 4wd
207441 rwd
207442 fwd
207443 4wd
207445 4wd
207446 fwd
207468 fwd
207469 4wd
207470 4wd
207471 4wd
207472 4wd
207473 fwd
207474 fwd
207477 fwd
207503 fwd
207563 4wd
207564 fwd
207565 fwd
207566 4wd
207567 4wd
207600 4wd
207604 4wd
207621 4wd
207633 fwd
207684 4wd
207705 fwd
207708 4wd
207714 fwd
207723 4wd
207725 fwd
207726 fwd
207727 rwd
207728 4wd
207753 fwd
207754 fwd
207761 4wd
207769 fwd
207770 fwd
207772 fwd
207773 fwd
207774 fwd
207779 4wd
207780 rwd
207781 rwd
207789 rwd
207801 4wd
207805 4wd
207830 4wd
207831 fwd

224382 4wd
224383 4wd
224386 4wd
224471 fwd
224472 4wd
224480 4wd
224487 4wd
224488 4wd
224490 4wd
224496 4wd
224510 4wd
224512 4wd
224513 4wd
224516 4wd
224522 4wd
224525 4wd
224526 4wd
224532 fwd
224534 4wd
224543 4wd
224546 4wd
224548 4wd
224550 4wd
224553 4wd
224570 4wd
224606 4wd
224608 4wd
224610 4wd
224652 4wd
224677 4wd
224678 4wd
224680 4wd
224694 4wd
224722 4wd
224725 4wd
224726 4wd
224743 4wd
224773 4wd
224776 4wd
224777 4wd
224798 4wd
224822 4wd
224828 4wd
224840 4wd
224843 4wd
224844 4wd
224850 4wd
224852 4wd
224854 4wd
224857 4wd
224858 4wd
224876 4wd
224881 4wd
224886 4wd
224888 4wd
224919 4wd
224921 4wd
224949 4wd
224955 4wd
224956 4wd
224984 4wd
224986 4wd
225002 4wd
225005 4wd
225010 4wd
225011 4wd
225053 4wd
225079 4wd
225081 4wd
225084 4wd
225088 4wd
225100 4wd
225121 4wd
225124 4wd
225125 4wd
225132 4wd
225159 4wd
225212 4wd
225214 4wd
225217 4wd
225222 4wd
225237 4wd
225793 4wd
225803 4wd
225895 4wd
225902 4wd
225904 4wd
225913 4wd
226008 4wd
226012 4wd
226014 4wd

306284 rwd
306285 4wd
306443 4wd
306444 4wd
306446 rwd
306595 4wd
306674 fwd
306719 rwd
306831 4wd
306865 4wd
306997 4wd
307208 rwd
307807 4wd
307810 4wd
307814 4wd
307864 4wd
308202 4wd
308214 4wd
308240 4wd
308241 4wd
308242 4wd
308275 4wd
308307 4wd
308325 4wd
308409 4wd
308441 4wd
308455 4wd
308490 4wd
308498 4wd
308506 4wd
308510 4wd
308556 4wd
308573 4wd
308693 4wd
308706 4wd
308765 4wd
308915 4wd
308926 4wd
308945 4wd
308947 4wd
308949 4wd
308951 4wd
309018 4wd
309042 4wd
309079 4wd
309117 4wd
309193 4wd
309194 4wd
309205 4wd
309210 4wd
309217 4wd
309242 4wd
309247 4wd
309254 4wd
309314 4wd
309349 4wd
309372 4wd
309552 4wd
309553 4wd
309587 rwd
309729 4wd
309757 4wd
309792 4wd
309927 4wd
310412 4wd
310490 4wd
310572 4wd
310916 4wd
310963 4wd
311181 fwd
311206 4wd
311217 fwd
311285 4wd
311288 4wd
311543 4wd
311648 fwd
311686 4wd
311882 4wd
311883 4wd
312100 4wd
312254 fwd
312289 fwd
312309 fwd
312322 4wd
312370 4wd
312372 4wd
312375 4wd
312376 4wd
312378 4wd
312379 4wd
312380 4wd

375430 4wd
375455 4wd
375483 4wd
375493 fwd
375509 4wd
375623 fwd
375653 rwd
375688 rwd
375724 fwd
375772 4wd
375795 4wd
375936 fwd
375977 fwd
375979 fwd
375983 fwd
375992 fwd
375999 rwd
376003 fwd
376050 rwd
376083 rwd
376109 rwd
376123 rwd
376139 rwd
376159 rwd
376208 rwd
376369 fwd
376429 fwd
376435 fwd
376602 4wd
376605 4wd
376656 rwd
376795 4wd
376844 rwd
376872 4wd
376887 4wd
376890 4wd
376891 4wd
376898 4wd
376904 4wd
376909 4wd
376910 4wd
376913 4wd
376921 4wd
376923 4wd
376924 4wd
376927 4wd
376936 4wd
376937 4wd
376938 4wd
376943 4wd
376947 4wd
376953 4wd
376960 4wd
376961 4wd
376970 4wd
376977 4wd
376984 4wd
376985 4wd
376996 4wd
376998 4wd
377011 4wd
377012 4wd
377015 4wd
377024 4wd
377028 4wd
377037 4wd
377040 rwd
377049 4wd
377051 4wd
377052 4wd
377054 4wd
377062 4wd
377065 4wd
377066 4wd
377075 4wd
377084 4wd
377087 4wd
377088 4wd
377100 4wd
377104 4wd
377114 4wd
377118 4wd
377122 4wd
377130 4wd
377137 4wd
377140 4wd
377141 4wd
377160 4wd
377170 4wd
377171 4wd
377186 4wd

In [45]:
driv_desc_completed = pd.DataFrame(index_nums6, columns=['index'])
driv_desc_completed['drive'] = d
driv_desc_completed_new=driv_desc_completed.drop_duplicates(subset=['index'])
driv_desc_completed_new.set_index('index',inplace=True )
driv_desc_completed_new.head()

,drive
index,
423,4wd
504,4wd
510,4wd
659,4wd
1038,4wd


In [46]:
df_yo.update(driv_desc_completed_new)
df_yo.isna().sum()

id                   0
url                  0
region               0
region_url           0
price                0
year                 0
manufacturer      7073
model             2530
condition         9894
cylinders        33444
fuel              1215
odometer           392
title_status      7004
transmission      1158
VIN             120074
drive            93792
size             91605
type              9966
paint_color     101144
image_url            0
description          2
state                0
lat               3283
long              3283
posting_date         0
dtype: int64

In [47]:
unique_size = df_limited_years['size'].unique()
size_list = list(unique_size)
size_list= size_list[1:]
size_list

['full-size', 'mid-size', 'compact', 'sub-compact']

In [48]:
size_desc=df_limited_years.loc[df_limited_years['size'].isna(), ['description','size', 'id']].copy()
size_desc['description']=size_desc['description'].str.lower().str.split()
size_desc.head()

,description,size,id
27,"[carvana, is, the, safer, way, to, buy, a, car...",NaN,7316814884
28,"[carvana, is, the, safer, way, to, buy, a, car...",NaN,7316814758
29,"[carvana, is, the, safer, way, to, buy, a, car...",NaN,7316814989
30,"[carvana, is, the, safer, way, to, buy, a, car...",NaN,7316743432
32,"[carvana, is, the, safer, way, to, buy, a, car...",NaN,7316343444


In [49]:
index_nums7=[]
s =[]
for index, row in size_desc.iterrows():
    for size in size_list:
        if size in row['description']:
            index_nums7.append(index)
            s.append(size)  
result7 = [None]*(len(index_nums7)+len(s))
result7[::2] = index_nums7
result7[1::2] =s

for x, y in zip(result7[::2], result7[1::2]):
    
    print(x,y)

248 compact
294 compact
583 compact
727 compact
874 full-size
1000 compact
1262 compact
1812 compact
1972 mid-size
1981 compact
2000 mid-size
2012 mid-size
2077 mid-size
2128 mid-size
2153 mid-size
2564 mid-size
2618 compact
2698 compact
2725 compact
2847 mid-size
2884 compact
3087 compact
3387 compact
3462 mid-size
3573 compact
3579 compact
5004 compact
5040 full-size
5185 mid-size
5189 full-size
5397 compact
5404 compact
5405 compact
5464 full-size
5483 compact
5532 compact
5616 compact
5621 compact
5675 compact
5693 full-size
5693 compact
5742 compact
5749 mid-size
5749 compact
5764 compact
5820 compact
5836 compact
5899 compact
5923 compact
5935 compact
5999 compact
6001 compact
6063 compact
6072 compact
6147 compact
6256 compact
6299 full-size
6299 compact
6307 full-size
6307 compact
6339 compact
6362 compact
6363 compact
6385 compact
6441 compact
6488 full-size
6520 full-size
6520 compact
6583 compact
6584 full-size
6605 compact
6614 full-size
6655 compact
6694 compact
6710 compa

169484 compact
169937 compact
172016 compact
173084 compact
173141 compact
176761 full-size
177876 full-size
178446 full-size
178674 mid-size
178674 compact
179838 mid-size
179838 compact
179845 mid-size
179845 compact
179873 full-size
181000 full-size
181015 full-size
181032 full-size
181033 full-size
181182 compact
181186 full-size
181188 full-size
182430 full-size
183662 full-size
185981 full-size
187188 compact
187196 compact
187458 full-size
187971 mid-size
187971 compact
189291 full-size
189470 compact
189620 full-size
191441 full-size
191755 compact
192555 compact
192996 full-size
193529 compact
200408 full-size
201000 compact
201246 full-size
202307 full-size
202942 full-size
205216 full-size
208174 compact
208426 compact
213784 mid-size
213784 compact
213973 mid-size
213973 compact
214465 full-size
214674 mid-size
214674 compact
215308 mid-size
215308 compact
215399 mid-size
215399 compact
215713 mid-size
215713 compact
216439 compact
216440 compact
216452 compact
216453 compa

408954 compact
408961 compact
408972 compact
408976 full-size
408987 compact
408989 full-size
408992 compact
409040 compact
409041 compact
409042 compact
409047 full-size
409064 compact
409119 full-size
409228 full-size
409231 compact
409263 full-size
409332 compact
409349 compact
409398 compact
409410 compact
409475 compact
409481 compact
409499 compact
409528 compact
409541 compact
409632 full-size
409640 full-size
409650 compact
409717 compact
409789 compact
409790 compact
409846 full-size
409860 compact
409869 compact
409897 compact
409937 full-size
410727 compact
410844 compact
410850 compact
410866 compact
410873 compact
410898 compact
410900 compact
410906 compact
410914 compact
410978 compact
410979 compact
410980 compact
410985 compact
410988 compact
410995 compact
411014 compact
411016 compact
411020 compact
411023 compact
411033 compact
411035 compact
411087 compact
411093 compact
411105 compact
411150 compact
411153 compact
411156 compact
411160 compact
411161 compact
41116

In [50]:
size_desc_completed = pd.DataFrame(index_nums7, columns=['index'])
size_desc_completed['size'] = s
size_desc_completed_new=size_desc_completed.drop_duplicates(subset=['index'])
size_desc_completed_new.set_index('index',inplace=True )
size_desc_completed_new.head()

,size
index,
248,compact
294,compact
583,compact
727,compact
874,full-size


In [51]:
df_yo.update(size_desc_completed_new)
df_yo.isna().sum()

id                   0
url                  0
region               0
region_url           0
price                0
year                 0
manufacturer      7073
model             2530
condition         9894
cylinders        33444
fuel              1215
odometer           392
title_status      7004
transmission      1158
VIN             120074
drive            93792
size             88753
type              9966
paint_color     101144
image_url            0
description          2
state                0
lat               3283
long              3283
posting_date         0
dtype: int64

In [ ]:
df_yo['type'].value_counts()

In [52]:
unique_type = df_limited_years['type'].unique()
type_list = list(unique_type)
# type_list= type_list[1:]
type_list = ['pickup',
 'truck',
 'other',
 'coupe',
 'SUV',
 'hatchback',
 'mini-van',
 'sedan',
 'offroad',
 'bus',
 'convertible',
 'wagon',
 'van']

In [53]:
type_desc=df_limited_years.loc[df_limited_years['type'].isna(), ['description','type', 'id']].copy()
type_desc['description']=type_desc['description'].str.lower().str.split()
type_desc.head()

,description,type,id
195,"[2012, black, 140-inch, dodge, charger, limous...",NaN,7316711193
196,"[2005, ford, focus, 189k, miles, automatic, tr...",NaN,7316707537
200,"[phone:, ☎, (205), 862-8003, text:, text, (205...",NaN,7316694884
201,"[phone:, ☎, (205), 862-8003, text:, text, (205...",NaN,7316694885
203,"[contemporary, automotive, address:, 3427, sky...",NaN,7316691449


In [54]:
index_nums8=[]
ty =[]
for index, row in type_desc.iterrows():
    for types in type_list:
        if types in row['description']:
            index_nums8.append(index)
            ty.append(types)  
result8 = [None]*(len(index_nums8)+len(ty))
result8[::2] = index_nums8
result8[1::2] =ty

for x, y in zip(result8[::2], result8[1::2]):
    
    print(x,y)

201 truck
204 pickup
204 truck
206 truck
220 truck
229 truck
235 truck
256 other
263 other
275 truck
283 truck
286 truck
288 pickup
288 truck
294 van
302 pickup
302 truck
347 convertible
349 truck
349 other
393 truck
394 other
395 other
396 other
397 other
423 truck
423 wagon
423 van
430 truck
433 truck
449 pickup
449 truck
459 truck
465 truck
477 truck
501 truck
502 truck
503 truck
504 pickup
504 truck
504 van
506 truck
507 truck
508 truck
510 pickup
510 truck
510 van
512 truck
513 truck
513 van
514 truck
515 truck
515 van
516 truck
517 truck
517 van
519 truck
519 van
521 truck
522 truck
523 pickup
523 truck
532 truck
549 truck
550 truck
551 truck
553 truck
555 truck
555 van
557 truck
560 sedan
569 pickup
569 truck
581 other
696 truck
696 bus
696 van
756 sedan
759 sedan
774 truck
789 truck
789 offroad
789 van
796 truck
796 offroad
796 van
799 truck
799 offroad
799 van
806 truck
806 offroad
806 van
810 truck
810 offroad
810 van
813 pickup
813 truck
813 other
813 offroad
813 van
821 pic

26629 truck
26630 truck
26632 truck
26640 van
26645 wagon
26645 van
26647 van
26648 truck
26650 van
26711 truck
26711 van
26795 truck
26795 van
26798 truck
26798 van
26799 truck
26799 van
26848 truck
26848 van
26851 truck
26851 van
26854 truck
26855 truck
26855 van
26858 truck
26861 truck
26861 van
27004 truck
27317 van
27359 truck
27359 offroad
27371 truck
27372 truck
27372 van
27470 truck
27470 van
27473 truck
27473 van
27474 truck
27477 truck
27477 van
27678 truck
27678 van
27679 truck
27679 van
27683 truck
27684 truck
27703 other
27777 truck
27777 sedan
27827 bus
27858 truck
27858 van
27872 truck
27926 truck
27926 van
27927 truck
27928 truck
27928 van
27929 truck
27931 truck
27931 van
27933 truck
27933 van
27934 truck
27934 van
28053 truck
28053 van
28060 pickup
28062 truck
28062 van
28063 truck
28063 van
28215 truck
28215 coupe
28219 van
28232 van
28263 wagon
28263 van
28315 wagon
28315 van
28323 truck
28323 van
28545 van
28546 van
28571 truck
28571 van
28693 sedan
28949 truck
291

89978 truck
90216 van
90237 other
90237 sedan
90517 truck
90891 truck
90922 truck
91403 truck
91403 offroad
91403 van
91418 truck
91418 offroad
91418 van
91426 truck
91426 offroad
91426 van
91427 truck
91427 offroad
91427 van
91537 truck
91669 sedan
91696 truck
91696 offroad
91696 van
91698 truck
91698 offroad
91698 van
91718 coupe
91734 truck
91734 offroad
91734 van
91739 van
91996 pickup
91996 truck
92054 pickup
92054 truck
92059 pickup
92059 truck
92218 pickup
92218 truck
92367 pickup
92367 truck
92458 pickup
92458 truck
92586 pickup
92586 truck
92644 pickup
92644 truck
92690 other
92749 pickup
92749 truck
92786 pickup
92786 truck
92857 sedan
92897 pickup
92897 truck
92934 pickup
92934 truck
92953 pickup
92953 truck
92956 pickup
92956 truck
92963 pickup
92963 truck
92964 pickup
92964 truck
93151 truck
93255 pickup
93255 truck
93439 van
93454 pickup
93454 truck
93578 pickup
93578 truck
93791 other
93914 truck
93914 van
93940 van
94113 truck
94113 van
94160 truck
94196 sedan
94347 tru

156043 other
156044 van
156091 truck
156091 van
156144 truck
156144 van
156177 truck
156177 sedan
156188 truck
156188 van
156205 truck
156205 other
156205 van
156227 truck
156227 van
156228 truck
156228 van
156256 sedan
156285 truck
156285 van
156324 other
156324 sedan
156339 pickup
156339 truck
156339 van
156340 pickup
156340 truck
156340 sedan
156390 pickup
156390 truck
156488 wagon
156503 truck
156567 pickup
156567 truck
156662 truck
156662 other
156663 truck
156664 pickup
156664 truck
156665 truck
156665 other
156666 truck
156666 other
156667 truck
156667 other
156716 convertible
156724 other
156724 van
156725 other
156750 other
156788 truck
156800 sedan
156800 wagon
156801 sedan
156908 truck
156918 sedan
156918 wagon
156934 sedan
156937 coupe
156939 pickup
156939 truck
156942 pickup
156942 truck
156993 sedan
157036 truck
157038 sedan
157073 coupe
157074 sedan
157121 sedan
157150 hatchback
157177 truck
157179 coupe
157476 van
157620 truck
157620 offroad
157620 van
157621 truck
1576

229693 other
229695 truck
229695 van
229711 truck
229711 van
229747 pickup
229747 truck
229747 sedan
229809 truck
229809 van
229816 truck
229816 van
229961 hatchback
230116 van
230123 truck
230123 van
230124 truck
230124 van
230192 truck
230192 van
230453 truck
230453 van
230455 truck
230455 van
230521 hatchback
230546 pickup
230546 truck
230546 other
230550 other
230551 other
230552 other
230553 other
230716 other
231392 pickup
231392 truck
231392 other
231395 other
231396 other
231397 other
231398 other
231536 truck
231613 truck
231659 truck
231741 truck
231741 van
231939 hatchback
231960 truck
231960 van
232314 truck
232397 truck
232449 truck
232449 van
232452 truck
232452 van
232673 other
232783 truck
232783 van
232802 truck
232803 truck
232817 pickup
232890 truck
233015 truck
233032 truck
233281 truck
233281 bus
233281 van
233467 wagon
233475 truck
233475 van
233483 truck
233483 van
233510 truck
233539 truck
233560 truck
233582 truck
233582 van
233655 truck
233655 other
233663 tru

316502 truck
316502 sedan
316503 pickup
316503 truck
316798 van
316818 truck
316818 sedan
316927 pickup
316927 truck
316929 pickup
316929 truck
316930 pickup
316930 truck
316935 pickup
316935 truck
316936 truck
316936 van
316939 truck
316939 hatchback
316941 pickup
316941 truck
316942 pickup
316942 truck
317507 wagon
317804 truck
317804 sedan
317855 pickup
317855 truck
317856 truck
317856 sedan
317857 truck
317857 sedan
317859 truck
317931 pickup
317931 truck
317987 van
318007 bus
318016 truck
318017 truck
318021 bus
318025 bus
318027 truck
318035 truck
318054 truck
318062 truck
318512 wagon
318628 other
318633 other
318635 other
318636 other
318639 other
318664 other
318779 truck
318802 truck
318986 truck
319199 truck
319307 pickup
319307 truck
319308 pickup
319308 truck
319311 truck
319317 pickup
319317 truck
319320 truck
319320 van
319555 truck
319556 pickup
319556 truck
319557 pickup
319557 truck
319558 pickup
319558 truck
319572 pickup
319572 truck
319857 pickup
319857 truck
31985

407303 pickup
407303 truck
407309 pickup
407309 truck
407392 pickup
407392 truck
407468 pickup
407468 truck
407744 pickup
407744 truck
407777 pickup
407777 truck
407880 pickup
407880 truck
407906 pickup
407906 truck
407998 pickup
407998 truck
408009 pickup
408009 truck
408213 sedan
408333 other
408403 convertible
408688 coupe
408688 hatchback
408688 sedan
408738 pickup
408738 truck
408738 other
408779 pickup
408779 truck
408798 pickup
408798 truck
408898 pickup
408898 truck
409119 pickup
409119 truck
409228 pickup
409228 truck
409263 pickup
409263 truck
409387 truck
409542 pickup
409542 truck
409846 pickup
409846 truck
409889 other
409937 pickup
409937 truck
409939 pickup
409939 truck
410653 other
410687 other
411306 truck
411306 hatchback
411306 offroad
412731 other
412824 van
413475 other
413607 truck
414110 truck
414110 offroad
414110 van
414974 sedan
414974 wagon
414975 sedan
414984 sedan
415016 truck
415027 sedan
415029 coupe
415045 coupe
415059 truck
415097 sedan
415129 truck
415

In [55]:
type_desc_completed = pd.DataFrame(index_nums8, columns=['index'])
type_desc_completed['type'] = ty
type_desc_completed_new=type_desc_completed.drop_duplicates(subset=['index'])
type_desc_completed_new.set_index('index',inplace=True )
type_desc_completed_new.head()

,type
index,
201,truck
204,pickup
206,truck
220,truck
229,truck


In [56]:
df_yo.update(type_desc_completed_new)
df_yo.isna().sum()

id                   0
url                  0
region               0
region_url           0
price                0
year                 0
manufacturer      7073
model             2530
condition         9894
cylinders        33444
fuel              1215
odometer           392
title_status      7004
transmission      1158
VIN             120074
drive            93792
size             88753
type              4586
paint_color     101144
image_url            0
description          2
state                0
lat               3283
long              3283
posting_date         0
dtype: int64

In [57]:
df_yo.drop(['id', 'url', 'region_url', 'VIN', 'image_url', 'description', 'lat', 'long', 'posting_date'], axis=1, inplace=True)
df_yo.head()

,region,price,year,manufacturer,model,condition,cylinders,fuel,odometer,title_status,transmission,drive,size,type,paint_color,state
27,auburn,33590,2014.0,gmc,sierra 1500 crew cab slt,good,8 cylinders,gas,57923.0,clean,other,NaN,NaN,pickup,white,al
28,auburn,22590,2010.0,chevrolet,silverado 1500,good,8 cylinders,gas,71229.0,clean,other,NaN,NaN,pickup,blue,al
29,auburn,39590,2020.0,chevrolet,silverado 1500 crew,good,8 cylinders,gas,19160.0,clean,other,NaN,NaN,pickup,red,al
30,auburn,30990,2017.0,toyota,tundra double cab sr,good,8 cylinders,gas,41124.0,clean,other,NaN,NaN,pickup,red,al
31,auburn,15000,2013.0,ford,f-150 xlt,excellent,6 cylinders,gas,128000.0,clean,automatic,rwd,full-size,truck,black,al


In [58]:
df_yo1 = df_yo.copy()
df_yo1.dropna(inplace=True)
df_yo1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 152023 entries, 31 to 426872
Data columns (total 16 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   region        152023 non-null  object 
 1   price         152023 non-null  int64  
 2   year          152023 non-null  float64
 3   manufacturer  152023 non-null  object 
 4   model         152023 non-null  object 
 5   condition     152023 non-null  object 
 6   cylinders     152023 non-null  object 
 7   fuel          152023 non-null  object 
 8   odometer      152023 non-null  float64
 9   title_status  152023 non-null  object 
 10  transmission  152023 non-null  object 
 11  drive         152023 non-null  object 
 12  size          152023 non-null  object 
 13  type          152023 non-null  object 
 14  paint_color   152023 non-null  object 
 15  state         152023 non-null  object 
dtypes: float64(2), int64(1), object(13)
memory usage: 19.7+ MB


In [64]:
import pickle
with open('cars.pkl', 'wb') as carpickle:
    pickle.dump(df_yo1,carpickle)

In [59]:
df_yo2=df_yo.copy()

In [60]:
df_yo2.drop(['drive', 'size', 'paint_color'], axis=1,inplace=True)
df_yo2.dropna(inplace=True)
df_yo2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 292719 entries, 27 to 426878
Data columns (total 13 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   region        292719 non-null  object 
 1   price         292719 non-null  int64  
 2   year          292719 non-null  float64
 3   manufacturer  292719 non-null  object 
 4   model         292719 non-null  object 
 5   condition     292719 non-null  object 
 6   cylinders     292719 non-null  object 
 7   fuel          292719 non-null  object 
 8   odometer      292719 non-null  float64
 9   title_status  292719 non-null  object 
 10  transmission  292719 non-null  object 
 11  type          292719 non-null  object 
 12  state         292719 non-null  object 
dtypes: float64(2), int64(1), object(10)
memory usage: 31.3+ MB


In [63]:
df_yo2.head()

,region,price,year,manufacturer,model,condition,cylinders,fuel,odometer,title_status,transmission,type,state
27,auburn,33590,2014.0,gmc,sierra 1500 crew cab slt,good,8 cylinders,gas,57923.0,clean,other,pickup,al
28,auburn,22590,2010.0,chevrolet,silverado 1500,good,8 cylinders,gas,71229.0,clean,other,pickup,al
29,auburn,39590,2020.0,chevrolet,silverado 1500 crew,good,8 cylinders,gas,19160.0,clean,other,pickup,al
30,auburn,30990,2017.0,toyota,tundra double cab sr,good,8 cylinders,gas,41124.0,clean,other,pickup,al
31,auburn,15000,2013.0,ford,f-150 xlt,excellent,6 cylinders,gas,128000.0,clean,automatic,truck,al


In [61]:
with open('cars_witthout_drive_size_paint_NaN.pkl', 'wb') as carpickles:
    pickle.dump(df_yo2,carpickles)

NameError: name 'pickle' is not defined

In [65]:
df_yo1['paint_color'].value_counts()

white     37949
black     30883
silver    23550
grey      16847
blue      15587
red       14557
custom     4249
brown      3388
green      2962
orange      847
yellow      839
purple      365
Name: paint_color, dtype: int64

In [62]:
df_yo1.head()

,region,price,year,manufacturer,model,condition,cylinders,fuel,odometer,title_status,transmission,drive,size,type,paint_color,state
31,auburn,15000,2013.0,ford,f-150 xlt,excellent,6 cylinders,gas,128000.0,clean,automatic,rwd,full-size,truck,black,al
55,auburn,19900,2004.0,ford,f250 super duty,good,8 cylinders,diesel,88000.0,clean,automatic,4wd,full-size,pickup,blue,al
59,auburn,14000,2012.0,honda,odyssey,excellent,6 cylinders,gas,95000.0,clean,automatic,fwd,full-size,mini-van,silver,al
73,auburn,15000,2017.0,dodge,charger rt 4dr sedan,excellent,8 cylinders,gas,90000.0,rebuilt,automatic,rwd,mid-size,sedan,grey,al
86,auburn,3000,2004.0,chrysler,town & country,good,6 cylinders,gas,176144.0,clean,automatic,fwd,mid-size,mini-van,silver,al


In [ ]:
# sample2 = manu_desc['description']

In [ ]:
# pattern = "([a-zA-Z]+(?:'[a-z]+)?)"
# tokenizer = RegexpTokenizer(pattern)
# sample_doc = tokenizer.tokenize(sample)
# sample_doc = [token.lower() for token in sample_doc]
# sw =stopwords.words('english')
# sample_doc = [token for token in sample_doc if token not in sw]

In [ ]:
# pattern_2 = "([a-zA-Z]+(?:'[a-z]+)?)"
# tokenizer = RegexpTokenizer(pattern_2)

In [ ]:
# pattern_2 = "([a-zA-Z]+(?:'[a-z]+)?)"
# tokenizer = RegexpTokenizer(pattern_2)
# manu_desc['tokenized_sents'] = manu_desc.apply(lambda row: tokenizer.tokenize(row['description']), axis=1)


In [ ]:
# manu_desc['new'] = manu_desc['tokenized_sents'].str.split()
# manu_desc.head()

In [ ]:
# manu_desc['description'] =manu_desc['description'].str.split()
# manu_desc.head()

In [ ]:
# manu_desc['description'] = manu_desc['description'].lower()
# manu_desc.head()

In [ ]:
# manu_desc['description'] = manu_desc['description'].str.lower()

In [ ]:
# manu_desc['exists'] = manu_desc.drop('manufacturer', 1).isin(manu_desc['manufacturer']).any(1) 
# manu_desc.exists.value_counts()

In [ ]:
# manu_desc["test"] = np.where(manu_desc["tokenized_sents"].isin(manufacturer), 'Other')


In [ ]:
# manu_desc['tokenized_sents'].lower()

In [ ]:
# car1 = set(unique_manufacturer) & set(manu_desc['tokenized_sents'][0])
# car1

In [ ]:
# set(manu_desc['tokenized_sents'][0])

In [ ]:
# df_new = manu_desc[manu_desc['tokenized_sents'].isin(['toyota'])]
# df_new.head()